In [1]:
import numpy as np
from pyscf import gto, scf, fci
from tabulate import tabulate
from PNOFs import HF_U2RDM, MU_U2RDM, GU_U2RDM, BBC1_U2RDM, HF_U2RDM_phys
from utils import DM2_DiagSum, twoDM_Eigvals, BST_FIDX, spectral_clean, twoDM_Eigvals_phys
import scipy 
import math

In [2]:
# Get Hydrogen in mo basis from PySCF

mol = gto.Mole()
mol.unit = 'B' 
L = 1.5
mol.atom = f"""
# He 0 0 0 
H   {-L}   {-L}   0.0
H    {L}   {-L}   0.0
H    {L}    {L}   0.0
H   {-L}    {L}   0.0
"""
# this basis has 2 functions for Helium
# mol.basis = "ccpvdz"
mol.basis = "sto-3g"
# mol.basis = "631g"

mol.spin =  0
mol.verbose= 0
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri_ao = mol.intor('int2e')
S = mol.intor('int1e_ovlp')

## Run Hartree-Fock.
mf = scf.UHF(mol)
mf.kernel()


# Harvest HF quantities

h1 = mf.get_hcore()
M = h1.shape[0]
C_a = mf.mo_coeff[0,:,:]
C_b = mf.mo_coeff[1,:,:]

h1_a = C_a.T@h1@C_a
h1_b = C_b.T@h1@C_b
h1_mo = (h1_a, h1_b)


h2_mo = BST_FIDX((eri_ao, eri_ao, eri_ao),S@C_a,S@C_b)




In [3]:
dm1_ao = mf.make_rdm1()
dm2_ao = mf.make_rdm2()

dm1_mo = dm1_ao.copy()
# # dm1_mo=C[:,0:N]@dm1_ao@C[:,0:N].T
dm1_mo[0]=(C_a.T@(S.T@dm1_mo[0]@S))@C_a
dm1_mo[1]=(C_a.T@(S.T@dm1_mo[1]@S))@C_a

dm2_mo = BST_FIDX(dm2_ao,S@C_a,S@C_b)

# P = C@C.T
# print(P)

occ_mo_a, occ_mo_b, C_mo_a, C_mo_b = spectral_clean(dm1_mo)
print(occ_mo_a, np.sum(occ_mo_a))
print(occ_mo_b, np.sum(occ_mo_b))


[1.0000000e+00 1.0000000e+00 1.1997763e-16 0.0000000e+00] 2.000000000000001
[1.00000000e+00 1.00000000e+00 2.95958516e-16 1.20360611e-16] 2.000000000000002


In [4]:
dm2_HF_mo = HF_U2RDM(occ_mo_a, occ_mo_b, mol.nao)
dm2_HF_mo_phys = HF_U2RDM_phys(occ_mo_a, occ_mo_b, mol.nao)

In [5]:
L_CONCERN = False
if L_CONCERN:
    for i in range(0,mol.nao):
        for j in range(0,mol.nao):
            for k in range(0,mol.nao):
                for l in range(0,mol.nao):
                    if not math.isclose(dm2_HF_mo[0][i,j,k,l], dm2_mo[0][i,j,k,l], abs_tol = 1e-10):
                        print(i,j,k,l, dm2_HF_mo[0][i,j,k,l], dm2_mo[0][i,j,k,l] )
    for i in range(0,mol.nao):
        for j in range(0,mol.nao):
            for k in range(0,mol.nao):
                for l in range(0,mol.nao):
                    if not math.isclose(dm2_HF_mo[1][i,j,k,l], dm2_mo[1][i,j,k,l], abs_tol = 1e-10):
                        print(i,j,k,l, dm2_HF_mo[1][i,j,k,l], dm2_mo[1][i,j,k,l])
    for i in range(0,mol.nao):
        for j in range(0,mol.nao):
            for k in range(0,mol.nao):
                for l in range(0,mol.nao):
                    if not math.isclose(dm2_HF_mo[2][i,j,k,l], dm2_mo[2][i,j,k,l], abs_tol = 1e-10):
                        print(i,j,k,l, dm2_HF_mo[2][i,j,k,l], dm2_mo[2][i,j,k,l] )

In [6]:
print(np.sum(twoDM_Eigvals(dm2_HF_mo)), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals_phys(dm2_HF_mo_phys)), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals(dm2_mo)), scipy.special.binom(4,2))

# print(np.sum(twoDM_Eigvals(dm2_HF_mo)), scipy.special.binom(4,2))
# print(np.sum(twoDM_Eigvals(dm2_mo)), scipy.special.binom(4,2))

12.00000000000002 6.0
12.00000000000002 6.0
12.000000000000014 6.0


In [7]:
# Run FCI  
cisolver = fci.direct_uhf.FCI()
cisolver.max_cycle = 100
cisolver.conv_tol = 1e-8
e_mo, ci_mo = cisolver.kernel(h1_mo, h2_mo, h1.shape[0], mol.nelec, ecore=mf.energy_nuc())
dm1_fci, dm2_fci  = cisolver.make_rdm12s(ci_mo, h1.shape[0], mol.nelec)
occ_mo_a, occ_mo_b, C_mo_a, C_mo_b = spectral_clean(dm1_fci)
print(occ_mo_a, np.sum(occ_mo_a))
print(occ_mo_b, np.sum(occ_mo_b))


[0.98295595 0.5        0.5        0.01704405] 2.0000000000000013
[0.98295595 0.5        0.5        0.01704405] 2.0000000000000004


In [8]:
dm2_HF_fci = HF_U2RDM(occ_mo_a, occ_mo_b, mol.nao)
dm2_HF_fci_phys = HF_U2RDM_phys(occ_mo_a, occ_mo_b, mol.nao)

In [9]:
print(np.sum(twoDM_Eigvals(dm2_HF_fci)), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals(dm2_fci)), scipy.special.binom(4,2))
print(np.sum(twoDM_Eigvals_phys(dm2_HF_fci_phys)), scipy.special.binom(4,2))

13.067014208521432 6.0
12.000000000000009 6.0
13.067014208521432 6.0
